# Advanced Tutorial: Named Entity Recognition using a Bi-LSTM with the Conditional Random Field Algorithm

Tutorial Link:  https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html

<img src="https://raw.githubusercontent.com/PythonWorkshop/intro-to-nlp-with-pytorch/master/images/bilstm_flow.png" width="80%">

The Bi-LSTM is trained on both past as well as future information from the given data as word embeddings or vectors representing the input words.

## Outline

* Definitions

    * Bi-LSTM
    * CRF and potentials
    * Viterbi

* Helper Functions

* Data

* Create the Network

* Train

* Evaluate

## Definitions


### Bi-LSTM (Bidirectional-Long Short-Term Memory)

As we saw, an LSTM addresses the vanishing gradient problem of the generic RNN by adding cell state and more non-linear activation function layers to pass on or attenuate signals to varying degrees.  However, the main limitation of an LSTM is that it can **only account for context from the past**, that is, the hidden state, h_t, takes only past information as input.

#### Named Entity Recognition Task

For the task of **Named Entity Recognition (NER)** it is helpful to have context from past as well as the future, or left and right contexts.  This can be addressed with a Bi-LSTM which is two LSTMs, one processing information in a forward fashion as we have already seen and another LSTM that processes the sequences in a reverse fashion giving the future context.  _That second LSTM is just reading the sentence in reverse._  

The hidden states from both LSTMs are then concatenated into a final output layer or vector.

### Conditional Random Field

We don't have to stop at the output vector from the Bi-LSTM!  We're not at our tag for the entity, yet.  We need to understand costs of moving from one tag to the next (or staying put on a tag, even).

In a CRF, we have the concept of a _transition matrix_ which is the costs associated with transitioning from one tag to another - a transition matrix is calculated/trained for each time step.  It is used in the determination of the best path through all potential sequences.

Say **B** is the tag for the beginning of an entity, **I** signifies that we are inside an entity (will follow a **B**) and **O** means we are outside an entity. 

Next, is an example of B-I-O scheme labeling for finding nouns in a sentence (by the way, there are a myriad of other schemes out there - see [Referenes](#references) for some more).

| Word | Scheme Tag |
| --- | --- |
| She | B |
| was | O |
| born | O |
| in | O |
| North | B |
| Carolina | I |
| but | O |
| grew | O |
| up | O |
| in | O |
| Texas | B |

Let's look at the transition matrix for the costs of moving from one tag (using our B-I-O scheme) to the next (remember our Bi-LSTM is understanding both the forward and reverse ordering to get more accurate boundaries for the named entities).

<img src="../images/crf_transition_matrix.png" width="60%">

The mathematical derivations for calculating this matrix and decoding it is beyond the scope of this notebook, however if you wish to learn more see [this](http://www.cs.columbia.edu/~mcollins/crf.pdf) article.

### Viterbi Algorithm

If each Bi-LSTM instance (time step) has an associated output feature map and CRF transition and emission values, then each of these time step outputs will need to be decoded into a path through potential tags and a final score determined.  This is the purpose of the Viterbi algorithm, here, which is commonly used in conjunction with CRFs.

Specifically, the Viterbi algorithm finds the optimal path through a sequence given a cost function by tracing backwards through a graph of all possible paths.  There are computational tricks to finding this path in the high dimensional space and we will see this in the code below (`_viterbi_decode`).

Here, let's see a simple example of just the Viterbi algorithm.  The simplicity of Viterbi is that at each time step, it "looks to the left" to find that best path and then moves to the right, repeating this "look to the left" until a "survivor path" or optimal path is found with the last column being the possible tags.  The score may also be found by tracing backwards along this path and using the metric decided upon.

In this example the optimal score (via a metric) is the lowest one, however, one could also look for the highest scoring path if another metric is used as is shown next.

<img src="https://raw.githubusercontent.com/PythonWorkshop/intro-to-nlp-with-pytorch/master/images/viterbi.png" width="70%">


Getting more realistic...

With regards to our NER work here, below is an example of a "survivor" path within the context of the linear-CRF where we are trying to find the highest scoring path through a sequence (giving us the tags and final score).  The transition matrix values are represented by the arrows and a sequence vector is shown as part of the overall cost function.

<img src="../images/linear_crf_example.png" width="70%">

### Putting it All Together

Here we have word embeddings as the data for the forward and reverse LSTMs.  The resulting forward vector (V_f) and backwards vector (V_b or Output layer, here) are concatenated into a final vector (V_o) that feeds into the CRF layer and is decoded via the Viterbi algorithm (part of CRF layer) into the optimal tag for that input.  Note, the initial values for the Hidden inputs for each LSTM (forward and reverse) are often a vector of random numbers.

<br><br>

<img src="https://raw.githubusercontent.com/PythonWorkshop/intro-to-nlp-with-pytorch/master/images/blstm_crf_details.png" width="70%">


<div  align="right"><a href="https://www.sciencedirect.com/science/article/pii/S1532046417300977">Reference</a></div>

> For a more in-depth discussion, see this excellent post describing the Bi-LSTM, CRF and usage of the Viterbi Algorithm (among other NER concepts and equations): [post](https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html).

**EXERCISE**:  What's wrong with the above diagram??

In [ ]:
import sys
{sys.executable}

**Make sure the PyTorch version here matches the `requirements.txt` file**

In [ ]:
import torch
torch.__version__

In [ ]:
# Imports for this tutorial
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import jdc
import json
from collections import defaultdict, OrderedDict
import math
import numpy as np

torch.manual_seed(1)

If we have a CUDA capable GPU, let's use it!  Initialize an object `device` to use later on tensors and models.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

## Helper Functions

EXERCISE:  Fill in the missing "blanks" (`_____`) with the correct code in a new cell.

```python
def argmax(vec):
    """Return the argmax as a python int"""
    _, idx = torch.max(vec, 1)
    return idx._____()


def prepare_sequence(seq, to_ix):
    """
    Input:
        seq - the sequence (array)
        to_ix - the indices to which seqence values are converted (dict)
        
    Output:
        Numerical tensor
        """
    idxs = [to_ix[w] for w in seq]
    return torch._____(idxs, dtype=torch.long)

def log_sum_exp(vec):
    """Compute log sum exp in a numerically stable way for 
    the forward algorithm"""
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score._____(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))
```

## Data

The data for this notebook came from GitHub <a href="https://github.com/tensorflow/tensorflow/issues" target="_blank">issues for Tensorflow</a> and <a href="https://github.com/pytorch/pytorch/issues" target="_blank">issues for PyTorch</a>- sentences in which _inline_ code was found.

**Problem statement**:  Predict in a corpus, what parts are Python code snippets - i.e. Python code is an entity we are trying to identify in text.

### Labeling data

**Install notes (tested on Mac Mojave)**:

* Install <a href="https://github.com/chakki-works/doccano#installation" target="_blank">doccano</a> into a Python environment (a <a href="https://docs.python.org/3.6/library/venv.html" target="_blank">virtual environment</a> for e.g.) (follow Option 2)
  * Create the virtual environment (recommended) or use a conda environment/native (Python 3.6+)
  * Make sure to _first_ install (before what's in `requirements.txt`:
    * <a href="http://postgresapp.com" target="blank">Postgres app</a>
    * `sudo pip install psycopg2` to install ps_config (needed by doccano install)
  * Then proceed with rest of local install (Option 2)
* Run the app according to the Usage instructions in the doccano Readme
  * NOTE:  `python3` may have to be substituted for using `python` command for using a `venv`
  * NOTE: there must not be any empty lines in import data
  
Labeled data in the `doccano` app will look similar to:

![](../images/doccano_label.png)

### Convert `doccano` exported data to training data format

`doccano` exports as `json` with one input datum plus labels per line where:
* `id` is the datum or corpus numerical id
* `text` is the issue snippet or corpus
* `annotations` are the labels and indices of each word/element in the single corpus - the `start_offset` and `end_offset` are used to find the words in the sentence.  This will get the data in the correct for training.

**Read the data file and print one line**

In [ ]:
# Read export file
with open('doccano_export.json', 'r') as f:
    lines = f.readlines()

print(json.loads(lines[1]))

**Split out words and labels**

In [ ]:
# The numerical doccano label to actual label (B-I-O scheme)
ix_to_label = {4: 'O', 3: 'I', 2: 'B'}
    
# train/test data
data = []

# Vocabulary
vocab = set()
    
# Loop over each data point (a corpus of labeled text) to extract words
for line in lines:
    # An ordered dict will keep items in order for further manipulation
    # so we initialize here
    orddict = OrderedDict({})
    # Lists to hold the words and labels
    words = []
    labels = []
    # Convert line to json
    injson = json.loads(line)
    annots = injson['annotations']
    text = injson['text']
    
    # Add each word annotation to OrderedDict
    for ann in annots:
        orddict[ann['start_offset']] = ann
    
    # Sort ordered dict because there's no guarantee reading json
    # maintained order
    orddict = sorted(orddict.items(), key=lambda x: x[1]['start_offset'])
    
    for item in orddict:
        # the item is a tuple where second value is the actual value we want
        ann = item[1]
        # Subset text string
        word = text[ann['start_offset']:(ann['end_offset'] + 1)].rstrip()
        label = ix_to_label[ann['label']]
        # Add to list for this datum/corpus
        words.append(word)
        labels.append(label)
        vocab.add(word)
    # Add to overall data containers
    data.append((words, labels))

**Split into train and test data**

In [ ]:
num_train = math.floor(len(data) * 0.8) # 80% to train
training_data, test_data = data[:num_train], data[num_train:]

## Train

### Create the Network

**Tags and hyperparameters**

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4
MINIBATCH_SIZE = 2
LEARNING_WEIGHT = 5e-2
WEIGHT_DECAY = 1e-4

In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        """Initialize network."""
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

In [ ]:
%%add_to BiLSTM_CRF

def init_hidden(self):
    """Two tensors to hold hidden states, one for each
    LSTM direction with dimensions of (num_layers, 
    minibatch, hidden_dim)"""
    # Minibatch small because small dataset below
    return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
            torch.randn(2, 1, self.hidden_dim // 2).to(device))


In [ ]:
%%add_to BiLSTM_CRF


def _forward_alg(self, feats):
    """Core magic of the Conditional Random Field.  
    
    Input:
        The word embeddeding vectors for a sentence
    
    Since we’re using PyTorch to compute gradients for us, 
    we technically only need the forward part of the forward-backward 
    algorithm """
    # Do the forward algorithm to compute the partition function
    init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
    # START_TAG ("<START>") has all of the score.
    init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

    forward_var = init_alphas

    # Iterate through the sentence
    for feat in feats:
        alphas_t = []  # The forward tensors at this timestep
        for next_tag in range(self.tagset_size):
            # broadcast the emission score: it is the same regardless of
            # the previous tag
            emit_score = feat[next_tag].view(
                1, -1).expand(1, self.tagset_size)
            # the ith entry of trans_score is the score of transitioning to
            # next_tag from i
            trans_score = self.transitions[next_tag].view(1, -1)
            # The ith entry of next_tag_var is the value for the
            # edge (i -> next_tag) before we do log-sum-exp
            next_tag_var = forward_var + trans_score + emit_score
            # The forward variable for this tag is log-sum-exp of all the
            # scores.
            alphas_t.append(log_sum_exp(next_tag_var).view(1))
        forward_var = torch.cat(alphas_t).view(1, -1)
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    alpha = log_sum_exp(terminal_var)
    return alpha

In [ ]:
%%add_to BiLSTM_CRF

def _get_lstm_features(self, sentence):
    """Compute output vector of BiLSTM - used in 
    the forward pass of network"""
    self.hidden = self.init_hidden()
    embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
    lstm_out, self.hidden = self.lstm(embeds, self.hidden)
    lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
    # Map LSTM features into tag space
    lstm_feats = self.hidden2tag(lstm_out)
    return lstm_feats

In [ ]:
%%add_to BiLSTM_CRF

def _score_sentence(self, feats, tags):
    """Gives the score of a provided tag sequence"""
    # Gives the score of a provided tag sequence
    score = torch.zeros(1).to(device)
    tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), 
                      tags])
    for i, feat in enumerate(feats):
        score = score + \
            self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
    score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
    return score

In [ ]:
%%add_to BiLSTM_CRF

def _viterbi_decode(self, feats):
    """Implements Viterbi algorithm for finding most likely sequence of labels.
    Used in the forward pass of the network.

    We take the maximum over the previous states as opposed to the sum. 
    Input:
        loglikelihoods: torch tensor.
    Output:
        tuple. The first entry is the loglikelihood of this sequence. The second is 
        the most likely sequence of labels. 
    """
    backpointers = []

    # Initialize the viterbi variables in log space
    init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
    init_vvars[0][self.tag_to_ix[START_TAG]] = 0

    # forward_var at step i holds the viterbi variables for step i-1
    forward_var = init_vvars
    for feat in feats:
        bptrs_t = []  # holds the backpointers for this step
        viterbivars_t = []  # holds the viterbi variables for this step

        for next_tag in range(self.tagset_size):
            # next_tag_var[i] holds the viterbi variable for tag i at the
            # previous step, plus the score of transitioning
            # from tag i to next_tag.
            # We don't include the emission scores here because the max
            # does not depend on them (we add them in below)
            next_tag_var = forward_var + self.transitions[next_tag]
            best_tag_id = argmax(next_tag_var)
            bptrs_t.append(best_tag_id)
            viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
        # Now add in the emission scores, and assign forward_var to the set
        # of viterbi variables we just computed
        forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1).to(device)
        backpointers.append(bptrs_t)

    # Transition to STOP_TAG
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    best_tag_id = argmax(terminal_var)
    path_score = terminal_var[0][best_tag_id]

    # Follow the back pointers to decode the best path.
    best_path = [best_tag_id]
    for bptrs_t in reversed(backpointers):
        best_tag_id = bptrs_t[best_tag_id]
        best_path.append(best_tag_id)
    # Pop off the start tag (we dont want to return that to the caller)
    start = best_path.pop()
    assert start == self.tag_to_ix[START_TAG]  # Sanity check
    best_path.reverse()
    return path_score, best_path

The following function explains the calculation of the scoring...

In [ ]:
%%add_to BiLSTM_CRF

def neg_log_likelihood(self, sentence, tags):
    """Calculate the negative log likelihood given a sequence and labels.
    This is used in training (only) because we don't need to create
    and check the B-I-O tags themselves - only the score is important
    here for calculating the loss."""
    feats = self._get_lstm_features(sentence)
    forward_score = self._forward_alg(feats)
    gold_score = self._score_sentence(feats, tags)
    return forward_score - gold_score

In [ ]:
%%add_to BiLSTM_CRF

def forward(self, sentence):
    """The forward pass function for training the network.
    This is used in inference only."""
    # Get the emission scores (output layer) from the 
    # BiLSTM 
    lstm_feats = self._get_lstm_features(sentence)

    # Find the best path, given the features.
    score, tag_seq = self._viterbi_decode(lstm_feats)
    return score, tag_seq

In [ ]:
def remove_punct(text):
    """Remove punctuation from a piece of text"""
    punct = list(".,()-")
    for p in punct:
        text = text.replace(p, '')
    return text
    
text = remove_punct(text)

In [ ]:
# Create a lookup dict for all possible words and record their index
word_to_ix = {k: v for (k, v) in zip(vocab, range(len(vocab)))}
tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}
ix_to_tag = {0: "B", 1: "I", 2: "O"}

**Initialize model and optimizer for training**

In [ ]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_WEIGHT, weight_decay=WEIGHT_DECAY)

**EXERCISE**:  Fill in the missing "blanks" (`_____`) with the correct code in a new cell.

```python
# Check predictions before training
with torch._____():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_sent = precheck_sent.to(device)
    pred =  model(precheck_sent)[1]
    print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
    print('Ground truth: ', training_data[0][1])
    print(training_data[0][0])
```

### Train model!

IMPORTANT NOTE:  If at workshop, please keep the number of epochs at or below 100 to save on the shared GPU usage.

In [ ]:
%%time
# Make sure prepare_sequence from earlier in the LSTM section is loaded
# again, normally you would do more than 300 epochs, but we have
# toy data

losses = []
for epoch in range(100):  
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance of LSTM
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
        sentence_in, targets = sentence_in.to(device), targets.to(device)

        # Step 3. A lot happens.  Run our forward pass to get features from BLSTM,
        # run the CRF and get the negative log likelihoods and find the best 
        # "path" through sentence with the tags using the viterbi algorithm 
        # (also part of forward pass).
        # BTW our dynamic computational graph is created with the forward pass
        # Returns the forward score - ground truth score (our loss measure)
        loss = model.neg_log_likelihood(sentence_in, targets)
        losses.append(loss.item())


        # Step 4. Compute the loss, gradients (backprop), and update the 
        # parameters by calling optimizer.step() - optimizer here is 
        # SGD for our CRF
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print("Epoch: {} Loss: {}".format(epoch+1, np.mean(losses)))

**Save model**

Doc:  https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [ ]:
torch.save(model.state_dict(), 'model_0.pth')

**EXERCISE**:  Change the optimization algorithm and retrain.  E.g. try the Adam optimizer.  

**Sanity check**

You must call `model.eval()` to set dropout and batch normalization layers to evaluation mode before running inference. Failing to do this will yield inconsistent inference results.

In [ ]:
model.eval()

# Sanity check for predictions after training
# No need to accumulate gradients because this is a validation
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[1][0], word_to_ix)
    precheck_sent = precheck_sent.to(device)
    pred =  model(precheck_sent)[1]
    print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
    print('Ground truth: ', training_data[1][1])
    print(training_data[1][0])

**Important Note**:  `with torch.no_grad()` is used so that gradients are not propagated which is important for when we are running predictions and scoring.

## Evaluate

Let's test our model on an unseen sentence.



In [ ]:
model.eval() # set again just in case

# Pick some test data
test_datum = test_data[0][0]
test_text = test_data[0][1]

with torch.no_grad():
    precheck_sent = prepare_sequence(test_datum, word_to_ix)
    precheck_sent = precheck_sent.to(device)
    pred =  model(precheck_sent)[1]
    print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
    print('Ground truth: ', test_text)
    print('Text: ', test_datum)

**EXERCISE**:  Calculate the average accuracy for the prediction above on all test data.

**EXERCISE**:   Select and print only the code snippets from above for the prediction as well as groud truth (actual labels) for a better picture of the predictions.

If the result wasn't very good, try more epochs to create a new trained model.

## References

1. [Understanding Bidirectional RNN in PyTorch](https://towardsdatascience.com/understanding-bidirectional-rnn-in-pytorch-5bd25a5dd66)
2. [Conditional Random Field Tutorial in PyTorch](https://towardsdatascience.com/conditional-random-field-tutorial-in-pytorch-ca0d04499463)
3. [Character-level neural network for biomedical named entity recognition](https://www.sciencedirect.com/science/article/pii/S1532046417300977)
4.  [Other named entity tag schemes](https://lingpipe-blog.com/2009/10/14/coding-chunkers-as-taggers-io-bio-bmewo-and-bmewo/)
5.  <a href="https://blog.algorithmia.com/introduction-to-optimizers/" target="_blank">Introduction to Optimizers</a>

**EXERCISE (Advanced)**:  collect copora (more GitHub issues?), label (with `doccano` or favorite tool) your own dataset and train and test (with code above)